# NDS#57

第57回 長岡IT開発者勉強会 (NDS#57) でデモした内容です。

## 試しにサンプルで学習

すでに事前学習済みのモデル VGG16 を使ってみます。

VGG16とは、Visual Geometory Groupが作成した畳み込みニューラルネットワークです。
ImageNetという、大規模画像データセットから画像を分類することをすでに学習されているモデルを使用することができます。

- 畳み込み層: 13層、プーリング層: 5層、全結合層: 3層
- 1000種類の画像分類

Kerasでは、有名な学習済みモデルを簡単に使うことができます。


In [ ]:
!python --version

In [ ]:
import keras

In [ ]:
import tensorflow
print(tensorflow.test.is_built_with_cuda())

## VGG16 を見てみよう

VGG16のモデル構成を見てみましょう。

In [ ]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np
from IPython.core.display import Image, display

In [ ]:
vgg16 = VGG16(weights = 'imagenet')

In [ ]:
vgg16.summary()

In [ ]:
# 予測用のメソッド
def predict(input_path):
  display(Image(input_path))   # 読み込んだ画像をnotebook上に表示
  img = image.load_img(input_path, target_size = (224, 224))    # 画像データを読み込み (VGG16のデフォルトは224x224のサイズ)
  
  x = image.img_to_array(img)   # 画像データを1次元配列に展開 (入力のため)
  x = np.expand_dims(x, axis = 0)   # 配列xの先頭 (axis=0) に新たな次元列を追加
  
  predicts = vgg16.predict(preprocess_input(x))
  
  top = 5
  results = decode_predictions(predicts, top = top)[0]
  
  for result in results:
    print(result)

In [ ]:
# predict('./data/sample/strawberry1.jpg')

# Google Colab用
from google.colab import files
uploaded = files.upload()
predict('strawberry1.jpg')

## 転移学習／ファインチューニングをしてみよう

今回は、以下の画像の分類を行います。

- 笹団子
- 柿の種
- イタリアン
    - みかづきとフレンドは分けていません
- バスセンターのカレー

画像はBing Search API v7 を使用して事前に取得します。取得した画像の中から、分類毎に50枚の学習データを用意します。


In [ ]:
import os
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Flatten, Dense, Dropout
from keras import optimizers
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import Image, display

In [ ]:
## 学習データの読み込み
## 事前に取得した学習データが格納されたzipファイル(train.zip)をアップロード
from google.colab import files
uploaded = files.upload()
!ls

In [ ]:
## 検証データの読み込み
## 事前に取得した検証データが格納されたzipファイル(validation.zip)をアップロード
from google.colab import files
uploaded = files.upload()
!ls

In [ ]:
## データ格納用ディレクトリ作成、zipファイルの展開
train_data_dir = './data/train'
validation_data_dir = './data/validation'
result_dir = './results'

if not os.path.exists('./data'):
    os.mkdir('./data')

if not os.path.exists(result_dir):
    os.mkdir(result_dir)

!mv train.zip validation.zip ./data
%cd ./data
!unzip train.zip
!unzip validation.zip
!ls
%cd -

In [ ]:
## 学習モデルの定義
img_rows = 150   # 画像のサイズ (横)
img_cols = 150    # 画像のサイズ (縦)
channels = 3    # 画像のチャンネル数 (RGBなので3チャンネル)

# 入力層の出力を定義
# (デフォルトは224×224×3だが、今回は150×150×3にしてみた)
input_tensor = Input(shape=(img_rows, img_cols, 3))

# 学習済みモデルの生成
# (include_top = False で、出力層側にある全結合層3つを含まない、weightsでimagenetのモデルを使用、input_tensorでlayers.Input()の出力を指定)
base_model = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

In [ ]:
# モデルを確認
base_model.summary()

In [ ]:
## 分類するクラスを定義
classes = ['buscenter_curry', 'italian', 'kakinotane', 'sasadango']
num_classes = len(classes)

## 出力層を作成
x = base_model.output    # ベースモデルの出力を層に入れる
x = Flatten(input_shape = base_model.output_shape[1:])(x)    # ベースモデルの出力を平滑化
x = Dense(256, activation = 'relu')(x)    # 全結合層 (活性化関数はrelu)
x = Dropout(0.5)(x)    # 0.5の確率でドロップアウト (過学習を防ぐため)
x = Dense(256, activation = 'relu')(x)    # 全結合層 (活性化関数はrelu)
x = Dropout(0.5)(x)    # 0.5の確率でドロップアウト (過学習を防ぐため)
predictions = Dense(num_classes, activation = 'softmax')(x)   # 全結合層 (最終出力のため、活性化関数はsoftmaxで4クラス分類をする)

In [ ]:
# ベースモデルに出力層を追加して学習モデルを作成する
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

In [ ]:
# modelの各層はリストに格納されている
model.layers

In [ ]:
# block5_conv1の直前 (block4_pool)までを凍結し、block5_conv1以降を学習する (ファインチューニング)
for layer in model.layers[:15]:
    layer.trainable = False

In [ ]:
# モデルをコンパイル
# 損失関数はカテゴリカルクロスエントロピー
# オプティマイザ (最適化アルゴリズム) は SGD (確率的勾配降下法)
# lr: 学習率, momentum: モーメンタム (慣性項: 前回の更新量にmomentum倍して、パラメータの更新を慣性的にする) 
# 指標は正答率 (accuracy)
model.compile(loss='categorical_crossentropy',
    optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
    metrics=['accuracy'])

In [ ]:
# 学習する画像データの前処理
# スケール変換の他、data augmentationで画像の水増しを行う
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,    # スケールを0..255段階から0..1へ変換 (0..1へ変換した方が学習の収束が早い)
    width_shift_range=0.1,  # ランダムに水平シフト
    height_shift_range=0.1,  # ランダムに垂直シフト
    shear_range=0.2,  # ランダムにシアー変換 (指定範囲内でのランダム角度で画像を引っ張る)
    zoom_range=0.2, #  ランダムに　ズーム
    rotation_range=5, # ランダムに回転する角度
    horizontal_flip=True # 水平方向に反転
)

In [ ]:
# テスト用の画像データの前処理
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [ ]:
batch_size = 8    # 1: オンライン学習(入力毎に誤差関数実行)、データ数n: バッチ学習(全データ読み込んで誤差関数実行)、1より大きくnより小さい: ミニバッチ (入力がバッチ数毎に誤差関数実行)

# 元画像から前処理した後のデータを使う
# KerasのImageDataGeneratorは、ディレクトリ構造から自動的にクラスラベルを識別してくれる (classes で指定した名前のディレクトリで識別)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

In [ ]:
# テストデータも同様に
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

In [ ]:
epochs = 50  # 学習回数 (全データ読み込みを何回繰り返すか)

# 学習
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//batch_size,   # エポック毎のインジケータステップ (200 / 8 = 25)
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples//batch_size
)

# loss: 予測された回答と正答との誤差 (小さくなれば良い)
# acc: 正答率 (大きくなれば良い)
# val_loss: テストデータでの誤差
# val_acc: テストデータでの正答率

In [ ]:
# 学習の履歴はhistoryに格納している
history.history

In [ ]:
# リストでは見にくいので、グラフ化する
# 誤差の推移を見る
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(range(epochs), loss, marker='.', label='loss')
plt.plot(range(epochs), val_loss, marker='.', label='val_loss')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
# 正答率の推移を見る
acc = history.history['acc']
val_acc = history.history['val_acc']

plt.plot(range(epochs), acc, marker='.', label='acc')
plt.plot(range(epochs), val_acc, marker='.', label='val_acc')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.show()

In [ ]:
# 学習結果をpickleで保存しておく
import pickle

# 学習履歴を保存
with open('./results/history1.pkl', mode='wb') as f:
    pickle.dump(history.history, f)

# 学習モデルを保存
model.save('./results/niigata-vgg16-finetuning.h5')

In [ ]:
# 学習履歴と学習モデルをローカルにダウンロード
from google.colab import files
files.download( "./results/history1.pkl" )

In [ ]:
files.download("./results/niigata-vgg16-finetuning.h5")

In [ ]:
# 学習モデルを使って推論をする

def predict(img_file_path):
    
    display(Image(img_file_path, width=150, unconfined=True))
    img = image.load_img(img_file_path, target_size=(img_rows, img_cols))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    # 学習時にImageDataGeneratorのrescaleで正規化したので同じ処理が必要！
    # これを忘れると結果がおかしくなるので注意
    x = x / 255.0

    # クラスを予測
    # 入力は1枚の画像なので[0]のみ
    pred = model.predict(x)[0]

    # 予測確率が高いトップ5を出力 (今回は4クラスしかないからあまり意味ないけど)
    top = 5
    top_indices = pred.argsort()[-top:][::-1]
    result = [(classes[i], pred[i]) for i in top_indices]
    for x in result:
        print(x)

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
predict('./italian1.jpg')